In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
from functools import reduce
import shutil, os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.model_pruning as pruning
from six.moves import cPickle as pickle
from six.moves import range

In [12]:
# Download pickle from: https://drive.google.com/file/d/1rDhbg-8J4ucd32WTY5Ff_oJMAY_-W0k7/view?usp=sharing

# Change to correct picke location.
pickle_file = '../../../CS4180_Deep_Learning/2_DeepLearningPreliminaries/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    
    del save  # hint to help gc free up memory
    
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)  # None does the same as np.newaxis
    
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [13]:
def accuracy(predictions, labels):
    
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [30]:
tf.reset_default_graph()

# Parameters.
batch_size = 10
kernel = [5, 5]
stride = [2, 2]
filters = 16
units_dense = 32

# Training data.
tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size, image_size, num_channels],
                                  name='train_data')
tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels], name='train_labels')

# Validation data.
tf_valid_dataset = tf.constant(valid_dataset)

# Test data.
tf_test_dataset = tf.constant(test_dataset)

def model(data, reuse=tf.AUTO_REUSE):
    
    with tf.variable_scope('model', reuse=reuse) as scope:
    
        # Convolution 1.
        conv1 = tf.layers.conv2d(data, filters, kernel, strides=stride, padding='same',
                                  activation=tf.nn.relu, name='conv1')

        # Convolution 2.
        conv2 = tf.layers.conv2d(conv1, filters, kernel, strides=stride, padding='same',
                                      activation=tf.nn.relu, name='conv2')

        # Dense.
        conv2_shape = conv2.shape.as_list()
        dense_units = reduce(lambda x, y: x * y, conv2_shape[1:])
        conv2_flat = tf.reshape(conv2, [-1, dense_units])    
        dense = tf.layers.dense(conv2_flat, units_dense, activation=tf.nn.relu, name='dense1')

        # Logits.
        logits = tf.layers.dense(dense, 10, name='dense2')
    
    return logits

# Training predictions.
with tf.name_scope('training'):
    
    # Training computation.
    logits = model(tf_train_dataset)

    # Sparse logit loss.
    loss = tf.losses.softmax_cross_entropy(tf_train_labels, logits)
    train_prediction = tf.nn.softmax(logits)

# Validation predictions.
with tf.name_scope('validation'):
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    
# Test predictions
with tf.name_scope('testing'):
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

# Optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate=0.002).minimize(loss, global_step=tf.train.get_global_step())

# Global step.
global_step = tf.train.get_or_create_global_step()

# Summaries.
tf.summary.scalar('loss', loss)
summary_op = tf.summary.merge_all()


### Training process ###


# Remove graphs dir.
if os.path.isdir('graphs/easy/regular'):
    shutil.rmtree('graphs/easy/regular')
    
# Training steps.
num_steps = 2001

with tf.Session() as sess:
    
    # Initialization.
    tf.global_variables_initializer().run()
    summary_writer = tf.summary.FileWriter('graphs/easy/regular', graph=sess.graph)
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, summary = sess.run([optimizer, loss, train_prediction, summary_op], feed_dict=feed_dict)
        
        if step % 200 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

            summary_writer.add_summary(summary, global_step=step)
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    
    summary_writer.close()


Minibatch loss at step 0: 2.323695
Minibatch accuracy: 0.0%
Validation accuracy: 14.2%
Minibatch loss at step 200: 0.567933
Minibatch accuracy: 80.0%
Validation accuracy: 79.4%
Minibatch loss at step 400: 0.063517
Minibatch accuracy: 100.0%
Validation accuracy: 82.3%
Minibatch loss at step 600: 0.387792
Minibatch accuracy: 90.0%
Validation accuracy: 83.6%
Minibatch loss at step 800: 0.800880
Minibatch accuracy: 80.0%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 1.054979
Minibatch accuracy: 80.0%
Validation accuracy: 84.5%
Minibatch loss at step 1200: 0.127083
Minibatch accuracy: 90.0%
Validation accuracy: 84.5%
Minibatch loss at step 1400: 0.987723
Minibatch accuracy: 70.0%
Validation accuracy: 85.0%
Minibatch loss at step 1600: 0.491490
Minibatch accuracy: 80.0%
Validation accuracy: 85.3%
Minibatch loss at step 1800: 1.594027
Minibatch accuracy: 70.0%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.560874
Minibatch accuracy: 70.0%
Validation accuracy: 85.8%
Test a